In [11]:
def suma(a:int, b: int) -> float:
    return a + b

print(suma(2,3))  # Output: 5
print(suma("Hola, ","mundo!"))  # Output: Hola, mundo!

print(suma.__annotations__)

5
Hola, mundo!
{'a': <class 'int'>, 'b': <class 'int'>, 'return': <class 'float'>}


In [21]:
class Persona:
    nombre: str
    edad: int
    
    def __init__(self, nombre: str, edad: int):
        self.nombre = nombre
        self.edad = edad

    def __repr__(self) -> str:
        return f"Persona(nombre={self.nombre}, edad={self.edad})"

print(Persona.__annotations__)

p = Persona("Carlos", 28)
print(f"Nombre: {p.nombre}, Edad: {p.edad}")
print(p)

{'nombre': <class 'str'>, 'edad': <class 'int'>}
Nombre: Carlos, Edad: 28
Persona(nombre=Carlos, edad=28)


In [31]:
from typing import Union, Optional

def sumar(a: Union[int, float], b: Optional[Union[int, float]]) -> float:
    if b is None:
        return a
    return a + b

print(sumar(5, 10))        # Output: 15
print(sumar(5.5, 4.5))     # Output: 10
print(sumar(5.5, None))    # Output: 5.5    

numeros: list[int] = [1, 2, 3, 4, 5]
print(numeros)  

palabras: list[str] = ["hola", "mundo", "python", "mundo", "javascript"]

contador: dict[str, int] = {}
for palabra in palabras:
    contador[palabra] = contador.get(palabra, 0) + 1

print(contador)
print(contador.values())


15
10.0
5.5
[1, 2, 3, 4, 5]
{'hola': 1, 'mundo': 2, 'python': 1, 'javascript': 1}
dict_values([1, 2, 1, 1])
{"nombre": "Ana", "edad": 25}


In [41]:
import json

a = Persona("Ana", 25)
j = {"nombre": a.nombre, "edad": a.edad}

print(json.dumps(j, indent=2))

{
  "nombre": "Ana",
  "edad": 25
}


In [59]:
# !pip install pydantic 
from pydantic import BaseModel, Field

class PersonaModel(BaseModel):
    nombre: str
    edad: int = Field(18, ge=18, le=65, description="Edad debe estar entre 18 y 65")


p1 = PersonaModel(nombre="Miguel", edad=25)
print(p1)
print(p1.model_dump_json(indent=2))

print(json.dumps(p1.model_dump(), indent=2))

nombre='Miguel' edad=25
{
  "nombre": "Miguel",
  "edad": 25
}
{
  "nombre": "Miguel",
  "edad": 25
}


In [62]:
print(json.dumps(p1.model_json_schema(), indent=2))

{
  "properties": {
    "nombre": {
      "title": "Nombre",
      "type": "string"
    },
    "edad": {
      "default": 18,
      "description": "Edad debe estar entre 18 y 65",
      "maximum": 65,
      "minimum": 18,
      "title": "Edad",
      "type": "integer"
    }
  },
  "required": [
    "nombre"
  ],
  "title": "PersonaModel",
  "type": "object"
}


## Modelo Producto con validación de fechas

Vamos a crear un modelo con fechas de disponibilidad y validaciones personalizadas.

In [63]:
from pydantic import BaseModel, Field, field_validator, model_validator
from datetime import datetime, timedelta
from typing import Optional

class Producto(BaseModel):
    nombre: str = Field(..., min_length=3, description="Nombre del producto")
    precio: float = Field(..., gt=0, description="Precio debe ser mayor a 0")
    desde: datetime = Field(default_factory=datetime.now, description="Fecha inicio de disponibilidad")
    hasta: Optional[datetime] = Field(None, description="Fecha de vencimiento")
    
    @model_validator(mode='after')
    def validar_fechas(self):
        """Valida que hasta sea mayor que desde, y si no existe, la crea con desde + 30 días"""
        # Si no se proporcionó 'hasta', la calculamos
        if self.hasta is None:
            self.hasta = self.desde + timedelta(days=30)
        
        # Validar que 'hasta' sea mayor que 'desde'
        if self.hasta <= self.desde:
            raise ValueError(f"La fecha 'hasta' ({self.hasta}) debe ser mayor que 'desde' ({self.desde})")
        
        return self
    
    def __repr__(self):
        return f"Producto(nombre='{self.nombre}', precio=${self.precio:.2f}, desde={self.desde.date()}, hasta={self.hasta.date()})"

print("✅ Clase Producto creada con validaciones")

✅ Clase Producto creada con validaciones


### Ejemplos de uso

Probemos diferentes casos: con fecha hasta, sin fecha hasta, y validación de fechas incorrectas.

In [64]:
# Ejemplo 1: Producto sin fecha 'hasta' (se calcula automáticamente)
print("1️⃣ Producto sin fecha 'hasta':")
p1 = Producto( nombre="Laptop", precio=1500.00, desde=datetime(2025, 1, 1) )
print(p1)
print(f"   Días de disponibilidad: {(p1.hasta - p1.desde).days}")
print()

1️⃣ Producto sin fecha 'hasta':
nombre='Laptop' precio=1500.0 desde=datetime.datetime(2025, 1, 1, 0, 0) hasta=datetime.datetime(2025, 1, 31, 0, 0)
   Días de disponibilidad: 30



In [65]:
# Ejemplo 2: Producto con ambas fechas especificadas
print("2️⃣ Producto con ambas fechas:")
p2 = Producto(
    nombre="Mouse Inalámbrico",
    precio=45.99,
    desde=datetime(2025, 2, 1),
    hasta=datetime(2025, 3, 15)
)
print(p2)
print(f"   Días de disponibilidad: {(p2.hasta - p2.desde).days}")
print()

2️⃣ Producto con ambas fechas:
nombre='Mouse Inalámbrico' precio=45.99 desde=datetime.datetime(2025, 2, 1, 0, 0) hasta=datetime.datetime(2025, 3, 15, 0, 0)
   Días de disponibilidad: 42



In [66]:
# Ejemplo 3: Validación - Fecha 'hasta' menor que 'desde' (ERROR)
print("3️⃣ Intentando crear producto con fechas inválidas:")
try:
    p3 = Producto(
        nombre="Teclado",
        precio=85.00,
        desde=datetime(2025, 3, 1),
        hasta=datetime(2025, 2, 15)  # ❌ Esta fecha es ANTES de 'desde'
    )
    print(p3)
except ValueError as e:
    print(f"   ❌ Error de validación: {e}")
print()

3️⃣ Intentando crear producto con fechas inválidas:
   ❌ Error de validación: 1 validation error for Producto
  Value error, La fecha 'hasta' (2025-02-15 00:00:00) debe ser mayor que 'desde' (2025-03-01 00:00:00) [type=value_error, input_value={'nombre': 'Teclado', 'pr...time(2025, 2, 15, 0, 0)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error



In [ ]:
# Ejemplo 4: Producto con fecha actual (sin especificar 'desde')
print("4️⃣ Producto con fecha actual:")
p4 = Producto( nombre="Monitor 24\"", precio=299.99 ) 

print(p4)
print(f"   Disponible desde: {p4.desde.strftime('%d/%m/%Y %H:%M')}")
print(f"   Disponible hasta: {p4.hasta.strftime('%d/%m/%Y %H:%M')}")
print(f"   Días de disponibilidad: {(p4.hasta - p4.desde).days}")
print()

4️⃣ Producto con fecha actual:
nombre='Monitor 24"' precio=299.99 desde=datetime.datetime(2025, 10, 21, 10, 59, 2, 813931) hasta=datetime.datetime(2025, 11, 20, 10, 59, 2, 813931)
   Disponible desde: 21/10/2025 10:59
   Disponible hasta: 20/11/2025 10:59
   Días de disponibilidad: 30



In [69]:
# Ejemplo 5: Serialización a JSON
print("5️⃣ Serialización del producto:")
print(p2.model_dump_json(indent=2))
print()

# Ver el esquema JSON del modelo
print("6️⃣ Esquema JSON del modelo Producto:")
print(json.dumps(Producto.model_json_schema(), indent=2))

5️⃣ Serialización del producto:
{
  "nombre": "Mouse Inalámbrico",
  "precio": 45.99,
  "desde": "2025-02-01T00:00:00",
  "hasta": "2025-03-15T00:00:00"
}

6️⃣ Esquema JSON del modelo Producto:
{
  "properties": {
    "nombre": {
      "description": "Nombre del producto",
      "minLength": 3,
      "title": "Nombre",
      "type": "string"
    },
    "precio": {
      "description": "Precio debe ser mayor a 0",
      "exclusiveMinimum": 0,
      "title": "Precio",
      "type": "number"
    },
    "desde": {
      "description": "Fecha inicio de disponibilidad",
      "format": "date-time",
      "title": "Desde",
      "type": "string"
    },
    "hasta": {
      "anyOf": [
        {
          "format": "date-time",
          "type": "string"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "description": "Fecha de vencimiento",
      "title": "Hasta"
    }
  },
  "required": [
    "nombre",
    "precio"
  ],
  "title": "Producto",
  "typ

In [72]:
class Empleado(BaseModel):
    persona: PersonaModel
    puesto: str = Field(..., min_length=3, description="Puesto de trabajo")
    salario: float = Field(..., gt=0, description="Salario debe ser mayor a 0")
    a_cargo: list[PersonaModel] = Field(default_factory=list, description="Lista de personas a cargo")

e1 = Empleado(
    persona=PersonaModel(nombre="Laura", edad=30),
    puesto="Desarrolladora",
    salario=75000.00,
    a_cargo=[PersonaModel(nombre="Juan", edad=23), PersonaModel(nombre="Sofía", edad=30)]
    
)
print(e1.model_dump_json(indent=2))

{
  "persona": {
    "nombre": "Laura",
    "edad": 30
  },
  "puesto": "Desarrolladora",
  "salario": 75000.0,
  "a_cargo": [
    {
      "nombre": "Juan",
      "edad": 23
    },
    {
      "nombre": "Sofía",
      "edad": 30
    }
  ]
}
